In [1]:
%load_ext autoreload
%autoreload 2

# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import scipy as sp
import wandb

import warnings

warnings.filterwarnings("ignore")
sys.path.append("../../")
device = "cuda" if torch.cuda.is_available() else "cpu"

from lightning_modules.jetGNN.submodels.interaction_gnn import InteractionGNN
from lightning_modules.jetGNN.submodels.gravnet import GravNet

In [23]:
# Load the config file
config_file = "jet_tag_config.yaml"
with open(config_file, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
model = InteractionGNN(config)

In [24]:
model = GravNet(config)

In [13]:
# For quick debugging
trainset, valset, testset = model.trainset, model.valset, model.testset

In [15]:
model.trainset, model.valset, model.testset = trainset, valset, testset

## Test Dataloading

In [25]:
model.setup(stage="fit")

Loading torch files
Mon Sep 19 06:00:21 2022
Building events
Mon Sep 19 06:04:21 2022
Defining figures of merit
Mon Sep 19 06:04:26 2022


In [22]:
sample = model.trainset[0]

In [23]:
sample

Data(x=[37, 4], y=0, log_pt=[37], log_E=[37], delta_eta=[37], delta_phi=[37], delta_pt=[37], delta_E=[37], delta_R=[37], edge_index=[2, 1332])

In [26]:
for batch in model.train_dataloader():
    print(batch)
    break

DataBatch(y=[200], pE=[9722], px=[9722], py=[9722], pz=[9722], log_pt=[9722], log_E=[9722], delta_pt=[9722], log_delta_pt=[9722], delta_E=[9722], log_delta_E=[9722], delta_R=[9722], delta_eta=[9722], delta_phi=[9722], jet_pt=[200], jet_pE=[200], jet_px=[200], jet_py=[200], jet_pz=[200], jet_mass=[200], jet_eta=[200], jet_phi=[200], x=[9722], batch=[9722], ptr=[201])


In [27]:
batch_idx = batch.batch

In [28]:
batch.jet_mass[batch_idx].shape

torch.Size([9722])

In [34]:
model.concat_feature_set(batch).shape

torch.Size([9722, 21])

## Train Model

In [16]:
logger = WandbLogger(
    project=config["project"], group="InitialTest", save_dir=config["artifacts"]
)
trainer = Trainer(
    gpus=1, max_epochs=config["max_epochs"], logger=logger
)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Defining figures of merit
Fri Sep 16 12:47:54 2022


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params
-----------------------------------------------
0 | feature_encoder | Sequential | 35.6 K
1 | grav_convs      | ModuleList | 363 K 
2 | output_network  | Sequential | 84.0 K
-----------------------------------------------
482 K     Trainable params
0         Non-trainable params
482 K     Total params
1.931     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 600/600 [00:43<00:00, 13.87it/s, loss=0.158, v_num=lobr]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 600/600 [00:43<00:00, 13.85it/s, loss=0.158, v_num=lobr]


: 